In [75]:
import pandas as pd
from sqlalchemy import create_engine
from config import username, password

In [105]:
csv_file = "Car_sales.csv"
car_sales_df = pd.read_csv(csv_file)

In [92]:
csv_file1 = "data.csv"
car_emissions_df = pd.read_csv(csv_file1)
car_emissions_df.head()

C:\Users\hish1\Anaconda\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (6,30) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,file,year,manufacturer,model,description,euro_standard,tax_band,transmission,transmission_type,engine_capacity,...,nox_emissions,thc_nox_emissions,particulates_emissions,fuel_cost_12000_miles,fuel_cost_6000_miles,standard_12_months,standard_6_months,first_year_12_months,first_year_6_months,date_of_change
0,DatapartC_july2000.csv,2000,Alfa Romeo,145 Range,1.6 Twin Spark 16v,2,NaN,M5,Manual,1598.0,...,NaN,NaN,NaN,NaN,618.0,NaN,NaN,NaN,NaN,NaN
1,DatapartC_july2000.csv,2000,Alfa Romeo,145 Range,1.8 Twin Spark 16v,2,NaN,M5,Manual,1747.0,...,NaN,NaN,NaN,NaN,633.0,NaN,NaN,NaN,NaN,NaN
2,DatapartC_july2000.csv,2000,Alfa Romeo,145 Range,Cloverleaf,2,NaN,M5,Manual,1970.0,...,NaN,NaN,NaN,NaN,663.0,NaN,NaN,NaN,NaN,NaN
3,DatapartC_july2000.csv,2000,Alfa Romeo,146 Range,1.6 Twin Spark 16v,2,NaN,M5,Manual,1598.0,...,NaN,NaN,NaN,NaN,626.0,NaN,NaN,NaN,NaN,NaN
4,DatapartC_july2000.csv,2000,Alfa Romeo,146 Range,1.8 Twin Spark 16v,2,NaN,M5,Manual,1747.0,...,NaN,NaN,NaN,NaN,633.0,NaN,NaN,NaN,NaN,NaN


In [106]:
new_car_sales_df = car_sales_df[['Manufacturer', 'Sales_in_thousands','Fuel_efficiency']].copy()

In [107]:
new_car_emissions_df = car_emissions_df[['manufacturer', 'co2', 'co_emissions']].copy()

In [108]:
new_car_emissions_df_1 = new_car_emissions_df.groupby('manufacturer')
new_car_emissions_df_group = new_car_emissions_df_1.mean()
car_emissions_group = new_car_emissions_df_group.reset_index()
car_emissions_group.head()

,manufacturer,co2,co_emissions
0,Abarth,151.789474,325.684211
1,Alfa Romeo,207.606452,480.092473
2,Aston Martin Lagonda,371.619048,513.705357
3,Audi,199.646015,593.322062
4,BMW,189.673802,311.442877


In [109]:
new_car_sales_df_1 = new_car_sales_df.groupby('Manufacturer')
new_car_sales_df_group = new_car_sales_df_1.mean() 
car_sales_group = new_car_sales_df_group.reset_index()
car_sales_group.head()

,Manufacturer,Sales_in_thousands,Fuel_efficiency
0,Acura,19.751250,25.250000
1,Audi,13.519000,23.333333
2,BMW,15.501667,25.000000
3,Buick,60.504750,24.250000
4,Cadillac,22.435600,20.600000


In [110]:
car_emissions_col = []
car_sales_col = [] 

column_names = ['manufacturer','co2_emissions','co_emissions']

for i in range(len(car_emissions_group)): 
    for j in range(len(car_sales_group)): 
        if car_emissions_group['manufacturer'][i] == car_sales_group['Manufacturer'][j]: 
            car_emissions_col.append([car_emissions_group['manufacturer'][i],
                            car_emissions_group['co2'][i],
                            car_emissions_group['co_emissions'][i], 
                           ])
            
            car_sales_col.append([car_emissions_group['manufacturer'][i], 
                                 car_sales_group['Sales_in_thousands'][j],
                                  car_sales_group['Fuel_efficiency'][j]])

car_efficiency_df = pd.DataFrame(car_emissions_col, columns=column_names )
car_efficiency_df.head()

,manufacturer,co2_emissions,co_emissions
0,Audi,199.646015,593.322062
1,BMW,189.673802,311.442877
2,Cadillac,254.779070,533.610465
3,Chevrolet,179.918736,489.557562
4,Dodge,212.365079,331.746032


In [111]:
column_names1 = ['manufacturer','sales_thousands','fuel_efficiency']
car_sales_df = pd.DataFrame(car_sales_col, columns=column_names1)
car_sales_df.head()

,manufacturer,sales_thousands,fuel_efficiency
0,Audi,13.519000,23.333333
1,BMW,15.501667,25.000000
2,Cadillac,22.435600,20.600000
3,Chevrolet,61.596111,28.444444
4,Dodge,82.740818,20.100000


In [99]:
from config import username, password

In [88]:
engine = create_engine(f'postgresql://<username>:<password>@localhost:5432/cars_DB')

In [101]:
engine.table_names()

['car_efficiency', 'car_sales']

### Importing to Postgres

In [112]:
car_efficiency_df.to_sql(name='car_efficiency', con=engine, if_exists='append', index=False)
car_sales_df.to_sql(name='car_sales', con=engine, if_exists='append', index=False)

In [113]:
pd.read_sql_query('select * from car_efficiency', con=engine).head()

,manufacturer,co2_emissions,co_emissions
0,Audi,199.646015,593.322062
1,BMW,189.673802,311.442877
2,Cadillac,254.779070,533.610465
3,Chevrolet,179.918736,489.557562
4,Dodge,212.365079,331.746032


In [114]:
pd.read_sql_query('select * from car_sales', con=engine).head()

,manufacturer,sales_thousands,fuel_efficiency
0,Audi,13.519000,23.333333
1,BMW,15.501667,25.000000
2,Cadillac,22.435600,20.600000
3,Chevrolet,61.596111,28.444444
4,Dodge,82.740818,20.100000
